#### Retriever

In [4]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Load reference doc
base_path = os.path.dirname(".")
ref_path = os.path.join(base_path, "../docs/explanation_guide.txt")


def load_docs():
    with open(ref_path, "r", encoding="utf-8") as f:
        raw = f.read()
        # split by blank lines
        paragraphs = [p.strip() for p in raw.split("\n\n") if p.strip()]
    return paragraphs


docs = load_docs()
vectorizer = TfidfVectorizer().fit(docs)
doc_vectors = vectorizer.transform(docs)


def retrieve_context(query, top_k=3):
    """Retrieve top K relevant chunks from the guide."""
    q_vec = vectorizer.transform([query])
    scores = cosine_similarity(q_vec, doc_vectors)[0]
    top_idx = scores.argsort()[-top_k:][::-1]
    return [docs[i] for i in top_idx]

#### Agent

In [5]:
import json


# Dummy LLM call (replace with OpenAI API)
def call_llm(prompt: str) -> str:
    return "[LLM Explanation Placeholder]\n" + prompt[:200] + "..."


# Optional helper to format key features
def top_features_string(features: dict, top_k=5):
    items = list(features.items())[:top_k]
    return "\n".join([f"- {k}: {v}" for k, v in items])


def interpret_prediction(prediction, score):
    if prediction == 1:
        return "High Risk"
    return "Low Risk"


def explain_prediction(features: dict, prediction: int, risk_score: float):
    label = interpret_prediction(prediction, risk_score)

    query = f"{label}, {', '.join(list(features.keys())[:3])}"

    retrieved = retrieve_context(query)

    prompt = f"""
    You are an AI banking risk analyst. Explain the model's decision.
    
    Model Prediction: {label} (risk score = {risk_score:.2f})

    Customer Features:
    {top_features_string(features)}

    Relevant Context:
    {json.dumps(retrieved, indent=2)}

    Write a 4-6 line explanation in simple, factual terms.
    """
    response = call_llm(prompt)

    return {
        "prediction": label,
        "risk_score": risk_score,
        "retrieved_context": retrieved,
        "explanation": response,
    }

#### Example Run

In [ ]:
# Example real row (replace with real dataset row)
features = {
"income": 4800,
"utilization": 0.82,
"missed_payments": 3,
"credit_history_length": 1.2,
"age": 24,
"debt_to_income": 0.54,
}


# todo: replace with actual model's predictions
prediction = 1
risk_score = 0.92


result = explain_prediction(features, prediction, risk_score)


print("=== MODEL OUTPUT ===")
print(result["prediction"], result["risk_score"])


print("\n=== RETRIEVED CONTEXT ===")
for c in result["retrieved_context"]:
    print("-", c)


print("\n=== FINAL EXPLANATION ===")
print(result["explanation"])

=== MODEL OUTPUT ===
High Risk 0.92

=== RETRIEVED CONTEXT ===
- Reasoning Patterns:
- Pattern A: High income + long history → low risk.
- Pattern B: High utilization + missed payments → high risk.
- Pattern C: Young age + short credit history → medium/high risk.
- Pattern D: Low utilization + no missed payments → low risk.
- General Risk Rules:
- Utilization above 70% significantly increases risk.
- More than 1 missed payment in last 12 months increases risk.
- A long clean credit history decreases risk.
- High income-to-debt ratio reduces risk.
- Younger customers with short histories may show more volatility.
- Feature Meanings:
- income: Monthly customer income. Higher income indicates more repayment capacity.
- utilization: Percentage of credit used relative to the limit.
- missed_payments: Count of missed or delayed payments.
- credit_history_length: Number of years since first credit account.
- age: Customer age in years.
- debt_to_income: Total debt divided by monthly income.

